#  🌍 Think global, Buy local : Analysis of Dataset content 🌍

In [45]:
# Add your imports here
import pandas as pd
import numpy as np
import scipy as sp
import json,folium,unidecode,branca
import branca.colormap as cm
import matplotlib.pyplot as plt
from geopy.distance import geodesic

%matplotlib inline

In [46]:
Importations_ = pd.read_csv("data/national-2016-import/National-2016-import/National_2016_Import.txt", sep=';', header=None,
                            names = ["Flux", "Mois", "Année", "CCPF6","A129","NC8","Pays","Valeur", "Masse","USUP"])

In [47]:
Importations_.head()

,Flux,Mois,Année,CCPF6,A129,NC8,Pays,Valeur,Masse,USUP
0,I,1,2016,011111,A01Z,10011100,AR,35,2,0
1,I,1,2016,011111,A01Z,10011100,AU,897,29,0
2,I,1,2016,011111,A01Z,10011100,BE,825,2292,0
3,I,1,2016,011111,A01Z,10011100,DE,781,2425,0
4,I,1,2016,011111,A01Z,10011100,ES,182964,343200,0


In [48]:
#Load dataset with country names in different languages
Countries_ = pd.read_csv("data/countries.csv")
countries = Countries_[["name","cca3","ccn3","cca2","latlng"]]
countries["name"] = countries.name.str.split(',').apply(lambda d:d[0])
countries.head(5)

C:\Users\Valentin\Anaconda3\envs\ada\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,cca3,ccn3,cca2,latlng
0,Aruba,ABW,533.0,AW,"12.5,-69.96666666"
1,Afghanistan,AFG,4.0,AF,"33,65"
2,Angola,AGO,24.0,AO,"-12.5,18.5"
3,Anguilla,AIA,660.0,AI,"18.25,-63.16666666"
4,Åland Islands,ALA,248.0,AX,"60.116667,19.9"


In [49]:
#Select the column with the latitude and longitude ('latlng') of each country
countries["loc"] = countries["latlng"].str.split(',')
countries["loc"] = countries["loc"].apply(lambda d: list(map(float, d)))

#collect the location of France
french_loc = countries[countries.cca3=="FRA"]["loc"].values[0]
french_loc

C:\Users\Valentin\Anaconda3\envs\ada\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Valentin\Anaconda3\envs\ada\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


[46.0, 2.0]

In [50]:
#function to calculate the geodesic distance between France and another country passed as an argument
distance_fr_udf = lambda x: int(geodesic(french_loc, x).km)
countries["dist_to_fr"] = countries["loc"].apply(distance_fr_udf) 
countries = countries.drop("latlng", axis=1)
countries.head(5)

C:\Users\Valentin\Anaconda3\envs\ada\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,name,cca3,ccn3,cca2,loc,dist_to_fr
0,Aruba,ABW,533.0,AW,"[12.5, -69.96666666]",7628
1,Afghanistan,AFG,4.0,AF,"[33.0, 65.0]",5458
2,Angola,AGO,24.0,AO,"[-12.5, 18.5]",6686
3,Anguilla,AIA,660.0,AI,"[18.25, -63.16666666]",6661
4,Åland Islands,ALA,248.0,AX,"[60.116667, 19.9]",1963


In [51]:
importations =Importations_.merge(countries[["name","cca2"]], 
                                  left_on=["Pays"], right_on=["cca2"], how="inner")\
                            .drop(["Pays", "USUP", "Flux"], axis=1)
importations.head(5)

,Mois,Année,CCPF6,A129,NC8,Valeur,Masse,name,cca2
0,1,2016,011111,A01Z,10011100,35,2,Argentina,AR
1,1,2016,011112,A01Z,10019120,1567,134,Argentina,AR
2,1,2016,011120,A01Z,10051018,164,2,Argentina,AR
3,1,2016,011120,A01Z,10059000,24895,70000,Argentina,AR
4,1,2016,011131,A01Z,10031000,35,2,Argentina,AR


In [52]:
countries_import = importations[["name", "Masse"]].groupby("name").sum()\
                                .merge(countries[["name","loc"]],
                                 left_index=True, right_on=["name"], how="inner")

In [53]:
countries_import.dtypes

Masse     int64
name     object
loc      object
dtype: object

In [57]:
countries_import.to_csv("data/countries_import.csv", index=False, float_format='%.3f')

In [86]:
NC8_ = pd.read_csv("data/national-2016-import/National-2016-import/Libelle_NC8_2016.txt", sep=";",
                    header = None, names = ["NC8", "Libellé", "Is_USUP", "Nature_USUP", "début_valid", "fin_valid"])

In [88]:
nc8 = NC8_.drop(["Is_USUP","Nature_USUP", "début_valid", "fin_valid"], axis=1)
nc8.head(5)

,NC8,Libellé
0,1012100,Chevaux reproducteurs de race pure
1,1012910,Chevaux destinés à la boucherie
2,1012990,Chevaux vivants (à l'exclusion des chevaux rep...
3,1013000,"Ânes, vivants"
4,1019000,"Mulets et bardots, vivants"


In [92]:
productions = pd.read_csv("data/productions.csv", index_col=0)

In [93]:
seasonnal_products = pd.read_csv("data/seasonnal_products.csv", index_col=0)

In [95]:
seasonnal_products_2 = pd.read_csv("data/seasonnal_products_2.csv", index_col=0)

In [134]:
seasonnal_products_2.head(5)

,Product,Season
0,Abeille,Printemps
1,Abeille,Et��
2,Abeille,Automne
3,Abeille,Hiver
4,Asperge,Printemps


In [106]:
meats_to_keep = ["Boeuf", "Chevre", "Vache", "Volaille", "Chapon","Cochon", "brebis", "vache", "annesse"]
meats = seasonnal_products_2[seasonnal_products_2["Product"].str.contains('|'.join(meats_to_keep))]

In [130]:
seasons_dict = {
    "Hiver" : ["Decembre", "Janvier", "Fevrier", "Mars"],
    "Printemps" : ["Mars", "Avril", "Mai", "Juin"],
    "Ete" : ["Juin", "Juillet", "Aout", "Septembre"],
    "Automne" : ["Septembre", "Octobre", "Novembre", "Decembre"]
}

In [133]:
df = pd.DataFrame.from_dict(seasons_dict, orient="index", columns = ["M0", "M1", "M2", "M3"])
seasons = df.assign(Months = df["M0"] + ',' + df["M1"] + ',' +df["M2"] + ',' + df["M3"])\
                .drop(["M0", "M1", "M2", "M3"], axis=1)
seasons.head(5)

,Months
Hiver,"Decembre,Janvier,Fevrier,Mars"
Printemps,"Mars,Avril,Mai,Juin"
Ete,"Juin,Juillet,Aout,Septembre"
Automne,"Septembre,Octobre,Novembre,Decembre"


In [139]:
meats.merge(seasons, left_on =["Season"], right_index=True, how="outer")

,Product,Season,Months
16,Boeuf,Printemps,"Mars,Avril,Mai,Juin"
20,"Boeuf, �� chaque morceau une cuisson",Printemps,"Mars,Avril,Mai,Juin"
51,Cochon,Printemps,"Mars,Avril,Mai,Juin"
98,Lait de brebis,Printemps,"Mars,Avril,Mai,Juin"
163,Vache �� viande,Printemps,"Mars,Avril,Mai,Juin"
167,Vache laiti��re,Printemps,"Mars,Avril,Mai,Juin"
171,Volaille,Printemps,"Mars,Avril,Mai,Juin"
175,Volailles de chair,Printemps,"Mars,Avril,Mai,Juin"
17,Boeuf,Et��,NaN
21,"Boeuf, �� chaque morceau une cuisson",Et��,NaN
